<a href="https://colab.research.google.com/github/vinhxtrinh/VinhTrinh_DTSC3020_Fall2025/blob/main/Assignment_6_WebScraping_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 6 (4 points) — Web Scraping

In this assignment you will complete **two questions**. The **deadline is posted on Canvas**.


## Assignment Guide (Read Me First)

- This notebook provides an **Install Required Libraries** cell and a **Common Imports & Polite Headers** cell. Run them first.
- Each question includes a **skeleton**. The skeleton is **not** a solution; it is a lightweight scaffold you may reuse.
- Under each skeleton you will find a **“Write your answer here”** code cell. Implement your scraping, cleaning, and saving logic there.
- When your code is complete, run the **Runner** cell to print a Top‑15 preview and save the CSV.
- Expected outputs:
  - **Q1:** `data_q1.csv` + Top‑15 sorted by the specified numeric column.
  - **Q2:** `data_q2.csv` + Top‑15 sorted by `points`.


In [13]:
#1) #Install Required Libraries
!pip -q install requests beautifulsoup4 lxml pandas
print("Dependencies installed.")


Dependencies installed.


### 2) Common Imports & Polite Headers

In [14]:
# Common Imports & Polite Headers
import re, sys, pandas as pd, requests
from bs4 import BeautifulSoup
HEADERS = {"User-Agent": (
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 "
    "(KHTML, like Gecko) Chrome/122.0 Safari/537.36")}
def fetch_html(url: str, timeout: int = 20) -> str:
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text
def flatten_headers(df: pd.DataFrame) -> pd.DataFrame:
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = [" ".join([str(x) for x in tup if str(x)!="nan"]).strip()
                      for tup in df.columns.values]
    else:
        df.columns = [str(c).strip() for c in df.columns]
    return df
print("Common helpers loaded.")


Common helpers loaded.


## Question 1 — IBAN Country Codes (table)
**URL:** https://www.iban.com/country-codes  
**Extract at least:** `Country`, `Alpha-2`, `Alpha-3`, `Numeric` (≥4 cols; you may add more)  
**Clean:** trim spaces; `Alpha-2/Alpha-3` → **UPPERCASE**; `Numeric` → **int** (nullable OK)  
**Output:** write **`data_q1.csv`** and **print a Top-15** sorted by `Numeric` (desc, no charts)  
**Deliverables:** notebook + `data_q1.csv` + short `README.md` (URL, steps, 1 limitation)

**Tip:** You can use `pandas.read_html(html)` to read tables and then pick one with ≥3 columns.


In [15]:
# --- Q1 Skeleton (fill the TODOs) ---
def q1_read_table(html: str) -> pd.DataFrame:
    """Return the first table with >= 3 columns from the HTML.
    TODO: implement with pd.read_html(html), pick a reasonable table, then flatten headers.
    """
    raise NotImplementedError("TODO: implement q1_read_table")

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean columns: strip, UPPER Alpha-2/Alpha-3, cast Numeric to int (nullable), drop invalids.
    TODO: implement cleaning steps.
    """
    raise NotImplementedError("TODO: implement q1_clean")

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort descending by Numeric and return Top-N.
    TODO: implement.
    """
    raise NotImplementedError("TODO: implement q1_sort_top")


In [21]:
import pandas as pd
import re

IBAN_URL = "https://www.iban.com/country-codes"

def _flatten_cols(cols):
    flat = []
    for c in cols:
        s = " ".join([str(x).strip() for x in (c if isinstance(c, tuple) else [c])])
        s = re.sub(r"\s+", " ", s).strip()
        flat.append(s)
    return flat

def q1_read_table(html: str) -> pd.DataFrame:
    tables = pd.read_html(html, flavor="bs4")
    chosen = None
    for t in tables:
        if t.shape[1] >= 3:
            chosen = t
            break
    chosen.columns = _flatten_cols(chosen.columns)
    low = {c: c.lower() for c in chosen.columns}
    colmap = {}
    for c in chosen.columns:
        lc = low[c]
        if "country" in lc and "alpha" not in lc:
            colmap["Country"] = c
        elif "alpha-2" in lc or "alpha 2" in lc:
            colmap["Alpha-2"] = c
        elif "alpha-3" in lc or "alpha 3" in lc:
            colmap["Alpha-3"] = c
        elif "numeric" in lc:
            colmap["Numeric"] = c
    df = chosen[[colmap["Country"], colmap["Alpha-2"], colmap["Alpha-3"], colmap["Numeric"]]]
    df.columns = ["Country", "Alpha-2", "Alpha-3", "Numeric"]
    return df

def q1_clean(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if out[c].dtype == "object":
            out[c] = out[c].astype(str).str.strip()
    out["Alpha-2"] = out["Alpha-2"].str.upper()
    out["Alpha-3"] = out["Alpha-3"].str.upper()
    out["Numeric"] = pd.to_numeric(out["Numeric"], errors="coerce").astype("Int64")
    out = out.dropna(subset=["Country"]).reset_index(drop=True)
    return out

def q1_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return (
        df.sort_values("Numeric", ascending=False, na_position="last")
          .head(top)
          .reset_index(drop=True)
    )

html = fetch_html(IBAN_URL)
raw = q1_read_table(html)
clean = q1_clean(raw)
top15 = q1_sort_top(clean, top=15)

display(top15)
top15.to_csv("data_q1.csv", index=False)


/tmp/ipython-input-3755037536.py:15: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(html, flavor="bs4")


,Country,Alpha-2,Alpha-3,Numeric
0,Zambia,ZM,ZMB,894
1,Yemen,YE,YEM,887
2,Samoa,WS,WSM,882
3,Wallis and Futuna,WF,WLF,876
4,Venezuela (Bolivarian Republic of),VE,VEN,862
5,Uzbekistan,UZ,UZB,860
6,Uruguay,UY,URY,858
7,Burkina Faso,BF,BFA,854
8,Virgin Islands (U.S.),VI,VIR,850
9,United States of America (the),US,USA,840


## Question 2 — Hacker News (front page)
**URL:** https://news.ycombinator.com/  
**Extract at least:** `rank`, `title`, `link`, `points`, `comments` (user optional)  
**Clean:** cast `points`/`comments`/`rank` → **int** (non-digits → 0), fill missing text fields  
**Output:** write **`data_q2.csv`** and **print a Top-15** sorted by `points` (desc, no charts)  
**Tip:** Each story is a `.athing` row; details (points/comments/user) are in the next `<tr>` with `.subtext`.


In [19]:
# --- Q2 Skeleton (fill the TODOs) ---
def q2_parse_items(html: str) -> pd.DataFrame:
    """Parse front page items into DataFrame columns:
       rank, title, link, points, comments, user (optional).
    TODO: implement with BeautifulSoup on '.athing' and its sibling '.subtext'.
    """
    raise NotImplementedError("TODO: implement q2_parse_items")

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Clean numeric fields and fill missing values.
    TODO: cast points/comments/rank to int (non-digits -> 0). Fill text fields.
    """
    raise NotImplementedError("TODO: implement q2_clean")

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    """Sort by points desc and return Top-N. TODO: implement."""
    raise NotImplementedError("TODO: implement q2_sort_top")


In [20]:
# Q2 — Write your answer here

from bs4 import BeautifulSoup
import pandas as pd
import re

HN_URL = "https://news.ycombinator.com/"

def _to_int(s):
    m = re.search(r"\d+", str(s))
    return int(m.group()) if m else 0

def q2_parse_items(html: str) -> pd.DataFrame:
    soup = BeautifulSoup(html, "lxml")
    rows = soup.select("tr.athing")
    items = []
    for r in rows:
        rank_txt = (r.select_one(".rank") or {}).get_text("", strip=True)
        rank = _to_int(rank_txt)

        a = r.select_one(".titleline a")
        title = a.get_text(strip=True) if a else ""
        link = a["href"] if a and a.has_attr("href") else ""

        sub = r.find_next_sibling("tr")
        subtext = sub.select_one(".subtext") if sub else None

        points_txt = subtext.select_one(".score").get_text(strip=True) if subtext and subtext.select_one(".score") else ""
        points = _to_int(points_txt)

        user = subtext.select_one(".hnuser").get_text(strip=True) if subtext and subtext.select_one(".hnuser") else ""

        comments = 0
        if subtext:
            comment_links = [a for a in subtext.select("a") if a.has_attr("href") and a["href"].startswith("item?id=")]
            if comment_links:
                comments = _to_int(comment_links[-1].get_text())

        items.append(
            {
                "rank": rank,
                "title": title,
                "link": link,
                "points": points,
                "comments": comments,
                "user": user,
            }
        )
    return pd.DataFrame(items)

def q2_clean(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for col in ["title", "link", "user"]:
        if col in out.columns:
            out[col] = out[col].fillna("").astype(str).str.strip()
    for col in ["points", "comments", "rank"]:
        if col in out.columns:
            out[col] = out[col].apply(_to_int).astype(int)
    return out

def q2_sort_top(df: pd.DataFrame, top: int = 15) -> pd.DataFrame:
    return (
        df.sort_values("points", ascending=False, kind="mergesort")
          .head(top)
          .reset_index(drop=True)
    )

html = fetch_html(HN_URL)
raw = q2_parse_items(html)
clean = q2_clean(raw)
top15 = q2_sort_top(clean, top=15)

display(top15)
top15.to_csv("data_q2.csv", index=False)



,rank,title,link,points,comments,user
0,9,Leaving Meta and PyTorch,https://soumith.ch/blog/2025-11-06-leaving-met...,652,158,saikatsg
1,29,Meta projected 10% of 2024 revenue came from s...,https://sherwood.news/tech/meta-projected-10-o...,567,440,donohoe
2,12,A Fond Farewell,https://www.farmersalmanac.com/fond-farewell-f...,544,196,erhuve
3,26,Rockstar employee shares account of the compan...,https://gtaforums.com/topic/1004182-rockstar-g...,379,239,mrzool
4,15,Denmark's government aims to ban access to soc...,https://apnews.com/article/denmark-social-medi...,302,214,c420
5,7,I Love OCaml,https://mccd.space/posts/ocaml-the-worlds-best/,252,160,art-w
6,27,OpenMW 0.50.0 Released – open-source Morrowind...,https://openmw.org/2025/openmw-0-50-0-released/,240,87,agluszak
7,30,We chose OCaml to write Stategraph,https://stategraph.dev/blog/why-we-chose-ocaml,135,100,lawnchair
8,8,James Watson has died,https://www.nytimes.com/2025/11/07/science/jam...,131,32,granzymes
9,21,PyTorch Helion,https://pytorch.org/blog/helion/,122,36,jarbus
